In [1]:
#help taken from anirudh topiwala git-repo

import random
import gym
import math
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
def preprocess_state(state):
    return np.reshape(state, [1, 4])

def replay(model,gamma,memory, batch_size):
    
    x_batch, y_batch = [], []
    #sample a random mini batch out of the memory stored in previous episode
    minibatch = random.sample(memory, min(len(memory), batch_size))
    for state, action, reward, next_state, done in minibatch:
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])

    model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
    

In [3]:
#  all the parameters needed
n_episodes=7000
n_win_ticks=200
# can have any number of steps per episode thus none
max_env_steps=None
gamma=0.99
epsilon=1.0 
epsilon_min=0.01
epsilon_decay=0.995
alpha=0.01
alpha_decay=0.01 
batch_size=64 
monitor=False 
quiet=False

# Model
model = Sequential()
model.add(Dense(24, input_dim=4, activation='relu'))
model.add(Dense(36, activation='relu'))
model.add(Dense(2, activation='linear'))
model.compile(loss='mse', optimizer=Adam(lr=alpha, decay=alpha_decay))


# environment setting 
memory = deque(maxlen=100000)
env = gym.make('CartPole-v1')
if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/cartpole-1', force=True)

scores = deque(maxlen=100)
solved = 0 
for e in range(n_episodes):
    state = preprocess_state(env.reset())
    done = False
    i = 0
    #run completely for one complete game for instance 
    #assuming some model for q given state and action initially the the model updates each time it runs
    while not done:
        env.render()
        
        #for exploration
        eps = max(epsilon_min, min(epsilon, 1.0 - math.log10((e + 1) * epsilon_decay)))
        action = env.action_space.sample() if (np.random.random() <= eps) else np.argmax(model.predict(state))
        
        #Next state prediction
        next_state, reward, done, _ = env.step(action)
        next_state = preprocess_state(next_state)
        memory.append((state, action, reward, next_state, done)) 
        state = next_state
        i += 1

    # score basically it the number of steps per episode it has survived thus has been assigned to i which is 
    # incremented every time step it is not done

    scores.append(i)
    mean_score = np.mean(scores)
    if mean_score >= n_win_ticks and e >= 100:
        if not quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
        solved = 1
        break
            
    if e % 100 == 0 and not quiet:
        print('[Episode {}] - Mean survival time over last 100 episodes was {} streaks.'.format(e, mean_score))

    replay(model,gamma,memory,batch_size)
    
    #reduce the epsilon over episode so that we reduce the exploration and increase exploitation over time 
    if epsilon > epsilon_min:
            epsilon *= epsilon_decay

if not quiet and solved == 0: print('Did not solve after {} episodes'.format(e))
env.close()


[Episode 0] - Mean survival time over last 100 episodes was 14.0 streaks.
[Episode 100] - Mean survival time over last 100 episodes was 10.86 streaks.
[Episode 200] - Mean survival time over last 100 episodes was 19.47 streaks.
[Episode 300] - Mean survival time over last 100 episodes was 22.07 streaks.
[Episode 400] - Mean survival time over last 100 episodes was 37.33 streaks.
[Episode 500] - Mean survival time over last 100 episodes was 57.92 streaks.
[Episode 600] - Mean survival time over last 100 episodes was 69.04 streaks.
[Episode 700] - Mean survival time over last 100 episodes was 89.77 streaks.
[Episode 800] - Mean survival time over last 100 episodes was 88.21 streaks.
[Episode 900] - Mean survival time over last 100 episodes was 138.46 streaks.
[Episode 1000] - Mean survival time over last 100 episodes was 139.54 streaks.
[Episode 1100] - Mean survival time over last 100 episodes was 126.37 streaks.
[Episode 1200] - Mean survival time over last 100 episodes was 159.32 stre

In [13]:
# render the model  
time = []
for i_episode in range(100):
    observation = env.reset()
    for t in range(1000):
        env.render()
        state = preprocess_state(observation)
        action = np.argmax(model.predict(state)[0])
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            time.append((t+1))
            break
print(np.mean(time))
env.close()

Episode finished after 316 timesteps
Episode finished after 201 timesteps
Episode finished after 90 timesteps
Episode finished after 232 timesteps
Episode finished after 405 timesteps
Episode finished after 82 timesteps
Episode finished after 269 timesteps
Episode finished after 256 timesteps
Episode finished after 186 timesteps
Episode finished after 88 timesteps
Episode finished after 92 timesteps
Episode finished after 84 timesteps
Episode finished after 192 timesteps
Episode finished after 213 timesteps
Episode finished after 114 timesteps
Episode finished after 248 timesteps
Episode finished after 218 timesteps
Episode finished after 78 timesteps
Episode finished after 294 timesteps
Episode finished after 96 timesteps
Episode finished after 105 timesteps
Episode finished after 301 timesteps
Episode finished after 168 timesteps
Episode finished after 245 timesteps
Episode finished after 98 timesteps
Episode finished after 200 timesteps
Episode finished after 128 timesteps
Episode f

In [14]:
print time

[316, 201, 90, 232, 405, 82, 269, 256, 186, 88, 92, 84, 192, 213, 114, 248, 218, 78, 294, 96, 105, 301, 168, 245, 98, 200, 128, 191, 109, 192, 181, 218, 96, 162, 89, 241, 480, 267, 134, 500, 91, 404, 103, 313, 104, 161, 169, 236, 88, 167, 120, 194, 122, 109, 102, 144, 500, 173, 290, 139, 500, 98, 83, 417, 283, 168, 163, 290, 103, 120, 110, 258, 92, 160, 225, 500, 271, 104, 184, 500, 131, 126, 110, 453, 149, 204, 100, 381, 167, 81, 93, 80, 230, 124, 94, 156, 252, 81, 142, 112]
